# ДЗ 4 Чтение номерных знаков

## Файзуллов Айрат

Работа с гугл диском

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
dataset_root_dir = 'drive/MyDrive/HW_DL_4'

Mounted at /content/drive


In [2]:
import zipfile
import os

with zipfile.ZipFile('drive/MyDrive/HW_DL_4/aimasters2023ml3hw4-license-plate-recognition.zip', 'r') as zip_ref:
    zip_ref.extractall('drive/MyDrive/HW_DL_4')

Создадим train dataset

In [3]:
import pandas as pd
from PIL import Image
from IPython.display import display
import zipfile
import os


data = {'image': [], 'name': []}

df = pd.DataFrame(data)

files = os.listdir('drive/MyDrive/HW_DL_4/train/train/simple')

for image_file in files:
  image_path = os.path.join('drive/MyDrive/HW_DL_4/train/train/simple', image_file)
  image = Image.open(image_path)
  df = pd.concat([df, pd.DataFrame({'image': [image], 'name': [image_file.split('_')[-1].split('.')[0]]})], ignore_index=True)

In [4]:
files = os.listdir('drive/MyDrive/HW_DL_4/train/train/complex')

for image_file in files:
  image_path = os.path.join('drive/MyDrive/HW_DL_4/train/train/complex', image_file)
  image = Image.open(image_path)
  df = pd.concat([df, pd.DataFrame({'image': [image], 'name': [image_file.split('_')[-1].split('.')[0]]})], ignore_index=True)

In [5]:
df_train = df

In [6]:
df_train

,image,name
0,<PIL.PngImagePlugin.PngImageFile image mode=RG...,B026CO90
1,<PIL.PngImagePlugin.PngImageFile image mode=RG...,C421PA35
2,<PIL.PngImagePlugin.PngImageFile image mode=RG...,B696TT35
3,<PIL.PngImagePlugin.PngImageFile image mode=RG...,B667AE35
4,<PIL.PngImagePlugin.PngImageFile image mode=RG...,Y487BY35
...,...,...
24542,<PIL.PngImagePlugin.PngImageFile image mode=RG...,O883BC799
24543,<PIL.PngImagePlugin.PngImageFile image mode=RG...,O930EP47
24544,<PIL.PngImagePlugin.PngImageFile image mode=RG...,T914MC35
24545,<PIL.PngImagePlugin.PngImageFile image mode=RG...,T409XE35


Создадим test dataset

In [7]:
data = {'image': [], 'name': []}

df = pd.DataFrame(data)

files = os.listdir('drive/MyDrive/HW_DL_4/test/result')

for image_file in files:
  image_path = os.path.join('drive/MyDrive/HW_DL_4/test/result', image_file)
  image = Image.open(image_path)
  df = pd.concat([df, pd.DataFrame({'image': [image], 'name': [image_file.split('.')[0]]})], ignore_index=True)

In [8]:
df_test = df
df_test['name'] = df_test['name'].astype(int)
df_test = df_test.sort_values(by='name', ascending=True)

In [10]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import tqdm
import json
import cv2
import os

from torch.nn            import Module, Sequential, Conv2d, AvgPool2d, GRU, Linear, LSTM
from torch.utils.data    import Dataset, DataLoader
from torch.nn.functional import ctc_loss, log_softmax
from torchvision         import models

from string import digits, ascii_uppercase

np.random.seed(42)

In [11]:
abc = "0123456789ABEKMHOPCTYX"

In [12]:
class RecognitionDataset(Dataset):
    """Class for training image-to-text mapping using CTC-Loss."""

    def __init__(self, config, alphabet=abc, transforms=None):
        """Constructor for class.

        Args:
            - config: List of items, each of which is a dict with keys "file" & "text".
            - alphabet: String of chars required for predicting.
            - transforms: Transformation for items, should accept and return dict with keys "image", "seq", "seq_len" & "text".
        """
        super(RecognitionDataset, self).__init__()
        self.config = config
        self.alphabet = alphabet
        self.image_names, self.texts = self._parse_root_()
        self.transforms = transforms

    def _parse_root_(self):
        image_names, texts = [], []
        for id, item in self.config:
            image_name = item['image']
            text = item['name']
            texts.append(text)
            image_names.append(image_name)
        return image_names, texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        """Returns dict with keys "image", "seq", "seq_len" & "text".
        Image is a numpy array, float32, [0, 1].
        Seq is list of integers.
        Seq_len is an integer.
        Text is a string.
        """
        image = np.array(self.image_names[item]).astype(np.float32) / 255.
        #print(image.shape)
        if len(image.shape) == 2:
          image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        if image.shape[2] == 4:
          image = image[:, : , :3]
        text = self.texts[item]
        seq = self.text_to_seq(text)
        seq_len = len(seq)
        output = dict(image=image, seq=seq, seq_len=seq_len, text=text)
        if self.transforms is not None:
            output = self.transforms(output)
        return output

    def text_to_seq(self, text):
        """Encode text to sequence of integers.

        Args:
            - String of text.

        Returns:
            List of integers where each number is index of corresponding characted in alphabet + 1.
        """

        # YOUR CODE HERE
        seq = [self.alphabet.find(c) + 1 for c in text]
        # END OF YOUR CODE

        return seq

In [13]:
class Resize(object):

    def __init__(self, size=(320, 64)):
        self.size = size

    def __call__(self, item):
        """Apply resizing.

        Args:
            - item: Dict with keys "image", "seq", "seq_len", "text".

        Returns:
            Dict with image resized to self.size.
        """

        # YOUR CODE HERE
        interpolation = cv2.INTER_AREA if self.size[0] < item["image"].shape[1] else cv2.INTER_LINEAR
        item["image"] = cv2.resize(item["image"], self.size, interpolation=interpolation)
        # END OF YOUR CODE

        return item

In [14]:
transforms = Resize(size=(320, 64))
dataset = RecognitionDataset(df_train.iterrows(), alphabet=abc, transforms=transforms)

In [19]:
for j in dataset:
  if j["image"].shape[2] != 3:
    print(j["image"].shape)
    print(j['text'])

In [20]:
x = dataset[0]
print(x.keys())

dict_keys(['image', 'seq', 'seq_len', 'text'])


In [21]:
def collate_fn(batch):
    """Function for torch.utils.data.Dataloader for batch collecting.

    Args:
        - batch: List of dataset __getitem__ return values (dicts).

    Returns:
        Dict with same keys but values are either torch.Tensors of batched images or sequences or so.
    """
    images, seqs, seq_lens, texts = [], [], [], []
    for item in batch:
      #print(item["image"].shape)
      images.append(torch.from_numpy(item["image"]).permute(2, 0, 1).float())
      seqs.extend(item["seq"])
      seq_lens.append(item["seq_len"])
      texts.append(item["text"])
    images = torch.stack(images)
    seqs = torch.Tensor(seqs).int()
    seq_lens = torch.Tensor(seq_lens).int()
    batch = {"image": images, "seq": seqs, "seq_len": seq_lens, "text": texts}
    return batch

In [22]:
xs = [dataset[i] for i in range(4)]
batch = collate_fn(xs)
print(batch.keys())

dict_keys(['image', 'seq', 'seq_len', 'text'])


In [23]:
for i in xs:
  print(i["image"].shape)

(64, 320, 3)
(64, 320, 3)
(64, 320, 3)
(64, 320, 3)


In [24]:
print("Image:", batch["image"].size())
print("Seq:", batch["seq"].size())
print("Seq:", batch["seq"])
print("Seq_len:", batch["seq_len"])
print("Text:", batch["text"])

Image: torch.Size([4, 3, 64, 320])
Seq: torch.Size([32])
Seq: tensor([12,  1,  3,  7, 19, 17, 10,  1, 19,  5,  3,  2, 18, 11,  4,  6, 12,  7,
        10,  7, 20, 20,  4,  6, 12,  7,  7,  8, 11, 13,  4,  6],
       dtype=torch.int32)
Seq_len: tensor([8, 8, 8, 8], dtype=torch.int32)
Text: ['B026CO90', 'C421PA35', 'B696TT35', 'B667AE35']


In [57]:
resnet = getattr(models, 'resnet18')(pretrained=True)
list(resnet.children())[:-3]

[Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False),
 Sequential(
   (0): BasicBlock(
     (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   )
   (1): BasicBlock(
     (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), pad

In [59]:
from torch.optim.lr_scheduler import CosineAnnealingLR

In [228]:
import torch.nn as nn

class FeatureExtractor(Module):

    def __init__(self, input_size=(64, 320), output_len=20):
        super(FeatureExtractor, self).__init__()

        h, w = input_size
        resnet = getattr(models, 'resnet34')(pretrained=True)
        self.cnn = Sequential(*list(resnet.children())[:-2])

        #self.custom_conv = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 2), padding=(3, 3), bias=False)
        self.pool = AvgPool2d(kernel_size=(h // 32, 1))
        self.proj = Conv2d(w // 32, output_len, kernel_size=1)

        self.num_output_features = self.cnn[-1][-1].bn2.num_features

    def apply_projection(self, x):
        """Use convolution to increase width of a features.

        Args:
            - x: Tensor of features (shaped B x C x H x W).

        Returns:
            New tensor of features (shaped B x C x H x W').
        """
        x = x.permute(0, 3, 2, 1).contiguous()
        x = self.proj(x)
        x = x.permute(0, 2, 3, 1).contiguous()

        return x

    def forward(self, x):
        # Apply conv layers
        #features = self.custom_conv(x)
        features = self.cnn(x)

        # Pool to make height == 1
        features = self.pool(features)

        # Apply projection to increase width
        features = self.apply_projection(features)

        return features


feature_extractor = FeatureExtractor()

x = torch.randn(1, 3, 64, 320)
y = feature_extractor(x)
print(y.size())
assert y.size() == (1, 1, 512, 20)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


torch.Size([1, 1, 512, 20])


In [229]:
feature_extractor = FeatureExtractor()

x = torch.randn(1, 3, 64, 320)
y = feature_extractor(x)
assert y.size() == (1, 1, 512, 20)

In [230]:
class SequencePredictor(Module):

    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.3, bidirectional=False):
        super(SequencePredictor, self).__init__()

        self.num_classes = num_classes
        self.rnn = GRU(input_size=input_size,
                       hidden_size=hidden_size,
                       num_layers=num_layers,
                       dropout=dropout,
                       bidirectional=bidirectional)

        fc_in = hidden_size if not bidirectional else 2 * hidden_size
        self.fc = Linear(in_features=fc_in,
                         out_features=num_classes)

    def _init_hidden(self, batch_size):
        """Initialize new tensor of zeroes for RNN hidden state.

        Args:
            - batch_size: Int size of batch

        Returns:
            Tensor of zeros shaped (num_layers * num_directions, batch, hidden_size).
        """
        num_directions = 2 if self.rnn.bidirectional else 1

        # YOUR CODE HERE
        h = torch.zeros(self.rnn.num_layers * num_directions, batch_size, self.rnn.hidden_size)
        # END OF YOUR CODE

        return h

    def _reshape_features(self, x):
        """Change dimensions of x to fit RNN expected input.

        Args:
            - x: Tensor x shaped (B x (C=1) x H x W).

        Returns:
            New tensor shaped (W x B x H).
        """

        # YOUR CODE HERE
        x = x.squeeze(1)
        x = x.permute(2, 0, 1)
        # END OF YOUR CODE

        return x

    def forward(self, x):
        x = self._reshape_features(x)

        batch_size = x.size(1)
        h_0 = self._init_hidden(batch_size)
        h_0 = h_0.to(x.device)
        x, h = self.rnn(x, h_0)

        x = self.fc(x)
        return x

In [231]:
sequence_predictor = SequencePredictor(input_size=512,
                                       hidden_size=128,
                                       num_layers=2,
                                       num_classes=len(abc) + 1)

In [232]:
x = torch.randn(1, 1, 512, 20)
assert sequence_predictor._reshape_features(x).size() == (20, 1, 512)

In [233]:
y = sequence_predictor(x)
assert y.size() == (20, 1, 23)

In [234]:
class CRNN(Module):

    def __init__(self, alphabet=abc,
                 cnn_input_size=(64, 320), cnn_output_len=20,
                 rnn_hidden_size=128, rnn_num_layers=2, rnn_dropout=0.3, rnn_bidirectional=False):
        super(CRNN, self).__init__()
        self.alphabet = alphabet
        self.features_extractor = FeatureExtractor(
            input_size=cnn_input_size, output_len=cnn_output_len
        )
        self.sequence_predictor = SequencePredictor(
            input_size=self.features_extractor.num_output_features,
            hidden_size=rnn_hidden_size, num_layers=rnn_num_layers,
            num_classes=len(alphabet)+1, dropout=rnn_dropout,
            bidirectional=rnn_bidirectional
        )

    def forward(self, x):
        features = self.features_extractor(x)
        sequence = self.sequence_predictor(features)
        return sequence

In [235]:
def pred_to_string(pred, abc):
    seq = []
    for i in range(len(pred)):
        label = np.argmax(pred[i])
        seq.append(label - 1)
    out = []
    for i in range(len(seq)):
        if len(out) == 0:
            if seq[i] != -1:
                out.append(seq[i])
        else:
            if seq[i] != -1 and seq[i] != seq[i - 1]:
                out.append(seq[i])
    out = ''.join([abc[c] for c in out])
    return out

def decode(pred, abc):
    pred = pred.permute(1, 0, 2).cpu().data.numpy()
    outputs = []
    for i in range(len(pred)):
        outputs.append(pred_to_string(pred[i], abc))
    return outputs

In [236]:
crnn = CRNN()

In [237]:
num_epochs = 30
batch_size = 32
num_workers = 4

In [238]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
crnn.to(device);

In [239]:
optimizer = torch.optim.Adam(crnn.parameters(), lr=3e-4, amsgrad=True, weight_decay=1e-4)

In [240]:
train_size = int(len(df_train) * 0.8)
config_train = df_train[:train_size]
config_val   = df_train[train_size:]

#train_dataset = RecognitionDataset(config_train.iterrows(), transforms=Resize())
train_dataset = RecognitionDataset(df_train.iterrows(), transforms=Resize())
val_dataset   = RecognitionDataset(config_val.iterrows(), transforms=Resize())

In [241]:
scheduler = CosineAnnealingLR(optimizer, T_max=int(len(train_dataset)))

In [242]:
train_dataloader = DataLoader(train_dataset,
                              batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True,
                              drop_last=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset,
                            batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True,
                            drop_last=False, collate_fn=collate_fn)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [243]:
ACTUALLY_TRAIN = True

In [226]:
crnn.train()
if ACTUALLY_TRAIN:
    for i, epoch in enumerate(range(num_epochs)):
      epoch_losses = []
      for j, b in enumerate(tqdm.tqdm(train_dataloader, total=len(train_dataloader))):
          images = b["image"].to(device)
          seqs_gt = b["seq"]
          seq_lens_gt = b["seq_len"]

          seqs_pred = crnn(images).cpu()
          log_probs = log_softmax(seqs_pred, dim=2)
          seq_lens_pred = torch.Tensor([seqs_pred.size(0)] * seqs_pred.size(1)).int()

          loss = ctc_loss(log_probs=log_probs,  # (T, N, C)
                          targets=seqs_gt,  # N, S or sum(target_lengths)
                          input_lengths=seq_lens_pred,  # N
                          target_lengths=seq_lens_gt)  # N

          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          scheduler.step()

          epoch_losses.append(loss.item())

      print(i, np.mean(epoch_losses))
else:
    image_train_log = cv2.imread("./resources/train_log.png")
    plt.figure(figsize=(15, 20))
    plt.imshow(image_train_log[:, :, ::-1], interpolation="bilinear")
    plt.axis("off")
    plt.show()

    with open("./data/crnn.pth.tar", "rb") as fp:
        state_dict = torch.load(fp, map_location="cpu")
    crnn.load_state_dict(state_dict)

100%|██████████| 767/767 [01:05<00:00, 11.71it/s]


0 1.737191776486388


100%|██████████| 767/767 [01:04<00:00, 11.84it/s]


1 0.5808532681350136


100%|██████████| 767/767 [01:05<00:00, 11.80it/s]


...

27 0.002278609645572361


100%|██████████| 767/767 [01:04<00:00, 11.86it/s]


28 0.002374612840257993


100%|██████████| 767/767 [01:04<00:00, 11.83it/s]

29 0.002242184244096279



100%|██████████| 767/767 [01:05<00:00, 11.74it/s]
0 0.00319003735637387

100%|██████████| 767/767 [01:05<00:00, 11.66it/s]
1 0.003010707575283857

100%|██████████| 767/767 [01:04<00:00, 11.92it/s]
2 0.0027783586740168357

100%|██████████| 767/767 [01:04<00:00, 11.85it/s]
3 0.0026729905792977054

100%|██████████| 767/767 [01:04<00:00, 11.90it/s]
4 0.0025629221327843283

100%|██████████| 767/767 [01:04<00:00, 11.87it/s]
5 0.0025050713490352345

100%|██████████| 767/767 [01:04<00:00, 11.87it/s]
6 0.002460858085213382

100%|██████████| 767/767 [01:04<00:00, 11.89it/s]
7 0.0023997828723787483

100%|██████████| 767/767 [01:04<00:00, 11.89it/s]
8 0.0023387975931881575

100%|██████████| 767/767 [01:04<00:00, 11.84it/s]9 0.002393255625260117

In [198]:
crnn.eval()
if ACTUALLY_TRAIN:
    val_losses = []
    for i, b in enumerate(tqdm.tqdm(val_dataloader, total=len(val_dataloader))):
        images = b["image"].to(device)
        seqs_gt = b["seq"]
        seq_lens_gt = b["seq_len"]

        with torch.no_grad():
            seqs_pred = crnn(images).cpu()
        log_probs = log_softmax(seqs_pred, dim=2)
        seq_lens_pred = torch.Tensor([seqs_pred.size(0)] * seqs_pred.size(1)).int()

        loss = ctc_loss(log_probs=log_probs,  # (T, N, C)
                        targets=seqs_gt,  # N, S or sum(target_lengths)
                        input_lengths=seq_lens_pred,  # N
                        target_lengths=seq_lens_gt)  # N

        val_losses.append(loss.item())

    print(np.mean(val_losses))
else:
    image_val_log = cv2.imread("./resources/val_log.png")
    plt.figure(figsize=(15, 20))
    plt.imshow(image_val_log[:, :, ::-1], interpolation="bilinear")
    plt.axis("off")
    plt.show()

100%|██████████| 154/154 [00:05<00:00, 26.52it/s]

0.0007382182399863256


100%|██████████| 154/154 [00:08<00:00, 18.14it/s]0.04778159772765569

100%|██████████| 154/154 [00:06<00:00, 24.91it/s]0.04428751703031216

100%|██████████| 154/154 [00:07<00:00, 21.42it/s]0.029348851702411888

Предсказываем для test

In [199]:
crnn.eval()

CRNN(
  (features_extractor): FeatureExtractor(
    (cnn): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(

In [200]:
df_test_copy = df_test.copy()
df_test_copy['name'] = df_test_copy['name'].astype(str)

In [201]:
test_dataset = RecognitionDataset(df_test_copy.iterrows(), transforms=Resize())

test_dataloader = DataLoader(test_dataset,
                            batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True,
                            drop_last=False, collate_fn=collate_fn)

In [202]:
answer_txt = []
for i in test_dataloader:
  images = i["image"]
  preds = crnn(images.to(device)).cpu().detach()
  texts_pred = decode(preds, crnn.alphabet)
  answer_txt.append(texts_pred)
  print(texts_pred)

['A904CC35', 'B581YY35', 'A481CC35', 'B161EP35', 'A725XO35', 'M417KX777', 'B611YY35', 'B960CC35', 'A119AO777', 'T295KP35', 'A676TO197', 'B721PA35', 'B378PB35', 'B399YX35', 'M347OP77', 'O329YX98', 'O977CC21', 'A763KP750', 'T717KM35', 'M959EB190', 'M806CM197', 'B004KP30', 'E354ET77', 'P349EK777', 'B736OK197', 'B303EY35', 'A963PE35', 'O032HC197', 'X688HC777', 'A001AA52', 'Y308KM47', 'E731AX35']
['B937CP777', 'M712YB190', 'B999HT28', 'B671YT77', 'K080BE50', 'P494KE197', 'M888OB777', 'T998TY35', 'M897BK178', 'B345YT35', 'E658CK777', 'M330YE197', 'B482TO150', 'B662YH777', 'C461TB199', 'A367KK77', 'E906PB777', 'E816OE750', 'C267TX35', 'X474YC197', 'Y271EX199', 'Y163BX35', 'X948XX35', 'B227OO178', 'E208PE32', 'T008OO35', 'O568CM47', 'E267TY777', 'Y334BO35', 'B141AP35', 'M813HE190', 'T409XB35']
['A604CA35', 'B320KX35', 'E385AX35', 'O830CP150', 'A849HA197', 'O925OT750', 'H333XE98', 'P846BK750', 'H071ME777', 'E886AX35', 'C032YC97', 'T861XC35', 'O459XH199', 'T784TT90', 'A364KY199', 'H447EM178', 'Y

In [203]:
from itertools import chain

flat_list = np.array(list(chain(*answer_txt)))
ind_lst = np.arange(len(flat_list))
#ind_lst = list(ind_lst)

In [204]:
flat_list.shape

(10518,)

In [205]:
len(ind_lst)

10518

In [206]:
columns = ['index', 'label']


df_res = pd.DataFrame(columns=columns)
df_res['index'] = ind_lst
df_res['label'] = flat_list

In [207]:
df_res = df_res.set_index('index')

In [208]:
from google.colab import files

csv_filename = 'my_data.csv'
df_res.to_csv(csv_filename, index=True)

files.download(csv_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>